# Customer Churn Prediction: Neural Network Implementation
This implements a custom Neural Network from scratch using NumPy to predict customer churn. It includes a complete preprocessing pipeline featuring log transformations, one-hot encoding, and SMOTE for handling class imbalance.

## Steps:
1. Import Libraries 
2. Read Data from `\Dataset` folder (Retrieved from UCI Repositories)
3. Clean Dataset (rename and format the column)
4. Data Preprocessing
5. Initialize the model
6. Train and test the data
7. Evaluate Performance using graph

### 0. Install Library Requirements

In [1]:
!pip install numpy pandas matplotlib seaborn tabulate imblearn


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1. Import Libraries
Import necessary libraries:
* `numpy`     : High performance array object
* `pandas`    : Data analysis and manipulation tools
* `matplotlib`: Data visualizations
* `seaborn`   : Data visualizations framework based on **matplotlib**
* `tabulate`  : Pretty-print tabular structure library
* `imblearn`  : SMOTE class balance

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
from imblearn.over_sampling import SMOTE
import seaborn as sns

#### 1.1 Helper Functions
* Handle mathematical operation used in the `NeuralNetwork` Class

In [3]:
def sigmoid(x): return 1 /(1 + np.exp(-x))
def derivative_sigmoid(sigmoid_x): return sigmoid_x * (1- sigmoid_x )
def relu(x): return np.maximum(0,x)
def derivative_relu(relu_x): return (relu_x>0).astype(float)
def binary_cross_entropy(y_true, y_pred, epsilon=1e-15):
    # Clip predictions to prevent log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

### 2. Read and Display Dataset Information
load the dataset and understand the data, and use the Interquartile Range (IQR) method to identify potential outliers in numerical columns.
2.1 Read dataset

In [4]:
def read_file(file_path: str):
    df = pd.read_csv(file_path)
    new_columns = [col.strip().replace('  ', ' ').replace(' ', '_').lower() for col in df.columns]
    df.columns = new_columns
    return df

path = r'..\Dataset\Customer Churn.csv'
df = read_file(path)
df.head(20)

,call_failure,complains,subscription_length,charge_amount,seconds_of_use,frequency_of_use,frequency_of_sms,distinct_called_numbers,age_group,tariff_plan,status,age,customer_value,churn
0,8,0,38,0,4370,71,5,17,3,1,1,30,197.640,0
1,0,0,39,0,318,5,7,4,2,1,2,25,46.035,0
2,10,0,37,0,2453,60,359,24,3,1,1,30,1536.520,0
3,10,0,38,0,4198,66,1,35,1,1,1,15,240.020,0
4,3,0,38,0,2393,58,2,33,1,1,1,15,145.805,0
5,11,0,38,1,3775,82,32,28,3,1,1,30,282.280,0
6,4,0,38,0,2360,39,285,18,3,1,1,30,1235.960,0
7,13,0,37,2,9115,121,144,43,3,1,1,30,945.440,0
8,7,0,38,0,13773,169,0,44,3,1,1,30,557.680,0
9,7,0,38,1,4515,83,2,25,3,1,1,30,191.920,0


2.2 Display dataset data types and null values

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   call_failure             3150 non-null   int64  
 1   complains                3150 non-null   int64  
 2   subscription_length      3150 non-null   int64  
 3   charge_amount            3150 non-null   int64  
 4   seconds_of_use           3150 non-null   int64  
 5   frequency_of_use         3150 non-null   int64  
 6   frequency_of_sms         3150 non-null   int64  
 7   distinct_called_numbers  3150 non-null   int64  
 8   age_group                3150 non-null   int64  
 9   tariff_plan              3150 non-null   int64  
 10  status                   3150 non-null   int64  
 11  age                      3150 non-null   int64  
 12  customer_value           3150 non-null   float64
 13  churn                    3150 non-null   int64  
dtypes: float64(1), int64(13)

2.3 Display dataset shape and its statisical values

In [6]:
df.describe()

,call_failure,complains,subscription_length,charge_amount,seconds_of_use,frequency_of_use,frequency_of_sms,distinct_called_numbers,age_group,tariff_plan,status,age,customer_value,churn
count,3150.000000,3150.000000,3150.000000,3150.000000,3150.000000,3150.000000,3150.000000,3150.000000,3150.000000,3150.000000,3150.000000,3150.000000,3150.000000,3150.000000
mean,7.627937,0.076508,32.541905,0.942857,4472.459683,69.460635,73.174921,23.509841,2.826032,1.077778,1.248254,30.998413,470.972916,0.157143
std,7.263886,0.265851,8.573482,1.521072,4197.908687,57.413308,112.237560,17.217337,0.892555,0.267864,0.432069,8.831095,517.015433,0.363993
min,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,15.000000,0.000000,0.000000
25%,1.000000,0.000000,30.000000,0.000000,1391.250000,27.000000,6.000000,10.000000,2.000000,1.000000,1.000000,25.000000,113.801250,0.000000
50%,6.000000,0.000000,35.000000,0.000000,2990.000000,54.000000,21.000000,21.000000,3.000000,1.000000,1.000000,30.000000,228.480000,0.000000
75%,12.000000,0.000000,38.000000,1.000000,6478.250000,95.000000,87.000000,34.000000,3.000000,1.000000,1.000000,30.000000,788.388750,0.000000
max,36.000000,1.000000,47.000000,10.000000,17090.000000,255.000000,522.000000,97.000000,5.000000,2.000000,2.000000,55.000000,2165.280000,1.000000


2.4. Detect Outliers

In [7]:
def detect_outliers_iqr(df, k=1.5):
    nums = df.select_dtypes(include='number')
    outlier_info = {}
    
    for c in nums.columns:
        #skip binary columns
        if(nums[c].nunique() <= 2):
            continue
        
        q1 = nums[c].quantile(0.25) #1st quartile
        q3 = nums[c].quantile(0.75) #3rd quartile
        iqr = q3 - q1
        lower = q1 - k * iqr
        upper = q3 + k * iqr
        mask = (nums[c] < lower) | (nums[c] > upper)
        outlier_info[c] = {
            'count': int(mask.sum()),
            'indices': nums.index[mask].tolist(),
            'lower': float(lower),
            'upper': float(upper)
        }
    return outlier_info

outliers = detect_outliers_iqr(df)
table_data = []
print('\nOutlier summary (IQR method):')
for col, info in outliers.items():
    if info['count'] > 0:
    # Calculate percentage
        perc = (info["count"] / len(df)) * 100
        
        # Add a list (row) to our table_data
        table_data.append([col, info["count"], f"{info['lower']:.3f}",
            f"{info['upper']:.3f}", f"{perc:.2f}%"])
headers = ["Column", "Outlier Count", "Lower Bound", "Upper Bound", "Percentage"]
print(tabulate(table_data, headers=headers))
    


Outlier summary (IQR method):
Column                     Outlier Count    Lower Bound    Upper Bound  Percentage
-----------------------  ---------------  -------------  -------------  ------------
call_failure                          47         -15.5           28.5   1.49%
subscription_length                  282          18             50     8.95%
charge_amount                        370          -1.5            2.5   11.75%
seconds_of_use                       200       -6239.25       14108.8   6.35%
frequency_of_use                     129         -75            197     4.10%
frequency_of_sms                     368        -115.5          208.5   11.68%
distinct_called_numbers               77         -26             70     2.44%
age_group                            170           0.5            4.5   5.40%
age                                  688          17.5           37.5   21.84%
customer_value                       116        -898.08        1800.27  3.68%


### 3. Data Cleaning 
This process take step before handling with outliers and data preprocessing.

3.1 Remove Duplicate Row (if any)

In [8]:
df = df.drop_duplicates()
print(f'\n[Changes] Removed duplicate rows. New shape={df.shape}\n')


[Changes] Removed duplicate rows. New shape=(2850, 14)



3.2 Remove Redundant Groups

`age_group` and `age` columns both have the same values but in different types, numeric and nominal respectively.
`age_group` column is dropped to prevent biases when learning

In [9]:
df = df.drop(columns=['age_group'])
print(f'\n[Changes] Dropped column: age_group due to redundancy. New shape={df.shape}\n\n')



[Changes] Dropped column: age_group due to redundancy. New shape=(2850, 13)




### 4. Data Preprocessing

The dataset will follows the exact steps to avoid any imbalance or data leakage during testing.

[Split] -> [Log] -> [Encode] -> [Fit] -> [Scale] -> [Balance]


#### 4.1 Split

* Splitting the dataset into train, and test sets, and both input,X and output,Y
* This function will reset the intialized index of X and Y dataset, and shuffle them before splitting to prevent **data leakage**

@function `split_data()`:
* params
    * `X`, `y` as input and output
    * `test_split` as testing split percentage ; default `0.2`
    * `randomness` as random values for randomize dataset index before splitting ; default `None`
* return
    * `X_train` = Training set without `Churn`
    * `Y_train` = `Churn` Training column
    * `Y_train` = Testing set without `Churn`
    * `Y_test` = `Churn` Testing column


In [10]:
def split_data(X, y, test_split=0.2, randomness=None):
    # Set seed for reproducibility
    if randomness is not None:
        np.random.seed(randomness)
    
    # reset X and Y current index
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    
    # Identify unique classes and their indices (0 and 1)
    unique_classes = np.unique(y)
    train_indices = []
    test_indices = []
    
    for cls in unique_classes:
        # Get indices of rows belonging to this class
        cls_indices = np.where(y == cls)[0]

        # Shuffle indices within this specific class
        np.random.shuffle(cls_indices)

        # Determine the split point
        total_count = len(cls_indices)
        test_count = int(total_count * test_split)
        
        # Split indices
        cls_test = cls_indices[:test_count]
        cls_train = cls_indices[test_count:]
        
        # Add to main lists
        test_indices.extend(cls_test)
        train_indices.extend(cls_train)
        
    # Shuffle the final combined indices so they aren't grouped by class
    np.random.shuffle(train_indices)
    np.random.shuffle(test_indices)
    
    # Use .iloc for DataFrames to select the rows
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    return X_train, X_test, y_train, y_test

X = df.drop(columns=['churn'], axis=1)
Y = df['churn']
X_train,X_test,y_train,y_test = split_data(X,Y,test_split=0.2, randomness=42)
print(f'[Changes] Successfully split data into Training and Testing.')


[Changes] Successfully split data into Training and Testing.


#### 4.2 Log Transformation

* This will treat the outliers found before, compressing all the values within the dataset to **reduce** the skewness of the data
* This step is to prevent unfair patterns.
* Use `numpy.log(1 + x)`
* log transformation on seperated train and test set to prevent **data leakage**

@function `log_transformation()`:
* params
    * `df_train`, `df_test` as training and testing dataset
    * `cols_log` as list of column to be log transformed
* return
    * `df_train` = train dataset that have been log transformed
    * `df_test` = test dataset that have been log transformed

In [11]:
def log_transformation(df_train, df_test, cols_log: list):
    for col in cols_log:
        df_train[col] = np.log1p(df_train[col])
        df_test[col] = np.log1p(df_test[col])
    print(f'[Changes] Applied log transformation to selected columns.')
    return df_train, df_test

cols_to_log = [
    'seconds_of_use',
    'frequency_of_use',
    'frequency_of_sms',
    'distinct_called_numbers',
    'call_failure',
    'customer_value',
    'charge_amount'
]
X_train, X_test = log_transformation(X_train,X_test,cols_to_log)
    

[Changes] Applied log transformation to selected columns.


#### 4.3 One-Hot Encoding

* Convert categorical column into multiple, numeric dummy columns that values between `0` and `1`.
* Example: `plan` have 3 categories  = `[cat1, cat2, cat3]`. It will split into dummy columns for each category, `plan_cat1`, `plan_cat2`, `plan_cat3` 
* use drop first column attribute to prevent **multicollinearity**

@function `get_train_category()`:
* params
    * `df` as dataset
    * `col_name` retrieve category from this column
* return
    * `list` = list of column name


@function `one_hot_encoding()`:
* params
    * `df` as dataset
    * `col_name` as column name that will apply one-hot encoding
    * `categories` as category inside `col_name`
    * `drop_first` drop first column to simplify the dummy column
* return
    * `converted_pd` = return the dataframe with the added dummy columns 

In [12]:
def get_train_categories(df, col_name):
    return sorted(list(set(df[col_name].tolist())))

def one_hot_encoding(df, col_name:str, categories, drop_first = True):
    data = df[col_name].tolist()
    
    active_cats = categories[1:] if drop_first and len(categories) > 1 else categories
    
    encoded_mtx = []
    for item in data:
        row = [0] * len(active_cats)
        if item in active_cats:
            index = active_cats.index(item)
            row[index] = 1
        encoded_mtx.append(row)
    
    #rename column for one hot encoded column
    new_cols = [f'{col_name}_{cat}' for cat in active_cats]
    #convert back to dataframe
    converted_pd  = pd.DataFrame(encoded_mtx, columns=new_cols,index=df.index)
    return converted_pd

train_encoded_parts = []
test_encoded_parts = []

categorical = ['complains', 'tariff_plan', 'status']
for col in categorical:
    train_categories = get_train_categories(X_train, col)
    train_encoded_parts.append(one_hot_encoding(X_train, col, train_categories, drop_first=True))        
    test_encoded_parts.append(one_hot_encoding(X_test, col, train_categories, drop_first=True))
print(f'[Changes] Applied one hot encoding to categorical columns')

#drop old column and join new columns
X_train = X_train.drop(columns=categorical).join(train_encoded_parts)
X_test = X_test.drop(columns=categorical).join(test_encoded_parts)

[Changes] Applied one hot encoding to categorical columns


#### 4.4 Scaling Feature

* Transforms features to fit within a specific range, usually `0`, by subtracting the minimum value and dividing by the range `(max - min)`
* Using `min()` and `max()` function to retrieve the minimum and maximum value in a column
* Scales the transformation on training data, and transform on train and test dataset to prevent data leakage

@function `get_scaling_params()`:
* params
    * `df_train` as training input dataset
* return
    * `param_dict` = a dictionary contains the minimum and maximum value for every key column

@function `min_max_transform()`:
* params
    * `df` as input dataset
    * `params` as dicitionary containing min and max value for each column
* return
    * `df_scaled` = scaled dataset 

In [17]:
def get_scaling_params(df_train):
    param_dict = {}
    for col in df_train.columns:   
        min_val = min(df_train[col])
        max_val = max(df_train[col])
        param_dict[col] = (min_val, max_val - min_val)
    return param_dict

#eg: min_max_transform(X_TEST, train_min_val, train_data_range)
def min_max_transform(df, params: dict):
    df_scaled = df.copy()

    for col, (min_v, data_range) in params.items():
        if data_range == 0:
            df_scaled[col] = 0.0 #base case float number
        else:
            df_scaled[col] = (df[col] - min_v) / data_range
            
    return df_scaled

X_train_scale_params = get_scaling_params(X_train)
X_train_scaled = min_max_transform(X_train, X_train_scale_params)
X_test_scaled = min_max_transform(X_test, X_train_scale_params)
print(f'[Changes] Applied Min Max Scaler on numerical columns')

[Changes] Applied Min Max Scaler on numerical columns


#### 4.5 Class Balancing

* The dataset contains around 80% `0` (Not Churn) and 20% `1` (Churn), causing imbalance and can cause the model to become biased, favor the majority class, and perform poorly on the crucial minority class.
* `SMOTE` is a **oversampling** technique used to create synthetic data using its algorithm to balance the class.
* Only apply to training data only to prevent data leakage
@function `fit_resample()`

In [ ]:
smote = SMOTE(sampling_strategy=0.8, random_state=42)
X_train_final, y_train_final = smote.fit_resample(X_train_scaled, y_train)
print(f"Before SMOTE - Class Distribution: {y_train.value_counts()}")
print(f"After SMOTE - Class Distribution: {pd.Series(y_train_final).value_counts()}")
print(f"Final training set size: {len(X_train_final)}")

### 5. Initialize the Model

#### 5.1 Intialize Variables
* `EPOCHS`      : Iteration counts
* `BATCH_SIZE`  : Hyperparameter of size of batch for smaller subsets 
* `PATIENCE`    : Hyperparameter stopping condition that wait for an improvement
* `input_dim` = input layer size (nodes)

In [ ]:
EPOCHS = 500
BATCH_SIZE = 32
PATIENCE = 100

input_dim = X_train_final.shape[1]

@Class `NeuralNetwork`
* A multi-layer perceptron (Backpropagation Neural Network) designed for binary classification.
* He initialization for weight stability, mini-batch gradient descent, and early stopping based on patience or error thresholds.

@function `feedforward()`:
* Feed the data forward, [Input] -> w1 -> [Hidden] -> w2 -> [Output]
* Input to hidden layer will use **ReLU** activation function to handle nonlinear
* Hidden layer to output layer will use **Sigmoid** activation function to **0 - 1** output

* params 
    * `X` as input dataset
* return
    * `output` = calculated output on input and weight , plus bias using respective activaatin function


@function `backpropagation()`:
* Calculates the gradients of the loss function with respect to each weight and bias in the network.
* Uses Gradient Clipping (max value of 5.0) to prevent exploding gradients and stability.
* Updates weight1, weight2, bias1, and bias2 using the calculated deltas and the learning rate (`alpha`).

* params
    * `X` as Input batch data.
    * `y` as Actual target labels
* output
    * Predicted values from the recent feedforward pass.


@function `calculate_accuracy()`
* Computes the percentage of correct predictions by comparing predicted probabilities against true labels
* 0.5 threshold to convert sigmoid probabilities into binary classes (0 or 1).

* params
    * `y_true`: Actual ground truth labels.
    * `y_pred_prob`: Predicted probabilities from the network.
* return
    * A float representing the accuracy score.

@function `train()`
* Manages the entire learning process over a set number of EPOCHS using Mini-batch Gradient Descent.
* Shuffles the training data every epoch to improve generalization and prevent the model from learning the order of the data.
* Stopping Condition:
    * `max_error`
    * `patience`

* params
    * `X_train`, `y_train`: Training features and labels
    * `X_test`, `y_test`: Validation/Testing features and labels used for tracking performance.
* return
    * `history`: A dictionary containing lists of train/test loss and accuracy for every epoch.

@function `predict()`
* Performs a final feedforward pass on unseen data to generate class predictions.
* Flattens the final output and applies a boolean threshold (0.5) to return integer class labels.
* params
    * `X`: The dataset to be predicted
* return
    * A NumPy array of binary predictions (`0` or `1`)

In [22]:
class NeuralNetwork:
    def __init__(self, input_dimension, hidden_nodes = 8, alpha = 0.01):
        self.alpha = alpha
        self.weight1 = np.random.randn(input_dimension, hidden_nodes) * np.sqrt(2/input_dimension) #He Initializaion keeps Weight stable
        self.weight2 = np.random.randn(hidden_nodes, 1) * np.sqrt(2/hidden_nodes)
        self.bias1 = np.zeros((1, hidden_nodes))
        self.bias2 = np.zeros((1, 1))
        
        self.train_loss, self.test_loss, self.train_acc, self.test_acc = [], [], [], []
        self.history = {
            'train_loss': self.train_loss,
            'test_loss': self.test_loss,
            'train_acc': self.train_acc,
            'test_acc': self.test_acc
        }    
        
    def feedforward(self,X):
        #Input -> Hidden
        self.hidden_input = X @ self.weight1 + self.bias1
        self.hidden_output = relu(self.hidden_input)
        #Hidden -> Output
        self.output_input = self.hidden_output @ self.weight2 + self.bias2
        self.output = sigmoid(self.output_input)
        
        return self.output
    
    def backpropagation(self, X, y, output):
        sample_num = X.shape[0]
        
        # output layer gradient Descent(BCE, Sigmoid)
        delta_output = (output - y)
        delta_hidden = np.dot(delta_output, self.weight2.T) * derivative_relu(self.hidden_output)
        
        #calc delta(Output: sigmoid , Hidden: ReLU)
        # Gradient Clipping (helps stability)
        max_grad = 5.0
        delta_output = np.clip(delta_output, -max_grad, max_grad)
        delta_hidden = np.clip(delta_hidden, -max_grad, max_grad)

        #update weights using gradient descent
        self.weight2 -= self.alpha * self.hidden_output.T @ delta_output / sample_num
        self.weight1 -= self.alpha * X.T @ delta_hidden / sample_num
        
        #update bias
        self.bias2 -= self.alpha * np.sum(delta_output, axis=0, keepdims=True) / sample_num
        self.bias1 -= self.alpha * np.sum(delta_hidden, axis=0, keepdims=True) / sample_num
    
    def calculate_accuracy(self, y_true, y_pred_prob):
        # Threshold at 0.5 for binary classification
        predictions = (y_pred_prob > 0.5).astype(int)
        correct = np.sum(predictions == y_true)
        return correct / len(y_true)
    
    def train(self, X_train, y_train, X_test, y_test):
        epochs = EPOCHS
        batch_size = BATCH_SIZE  # 32
        max_error = 0.01
        best_loss = float('inf')
        patience_count = 0
        patience = PATIENCE
        
        X_tr = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
        y_tr = np.array(y_train).reshape(-1, 1)
        X_te = X_test.values if isinstance(X_test, pd.DataFrame) else X_test
        y_te = np.array(y_test).reshape(-1, 1)
        
        n_samples = X_tr.shape[0]
        
        for epoch in range(epochs):
            # Shuffle data each epoch
            indices = np.random.permutation(n_samples)
            X_tr_shuffled = X_tr[indices]
            y_tr_shuffled = y_tr[indices]
            
            # Mini-batch training
            for i in range(0, n_samples, batch_size):
                batch_X = X_tr_shuffled[i:i+batch_size]
                batch_y = y_tr_shuffled[i:i+batch_size]
                
                output = self.feedforward(batch_X)
                self.backpropagation(batch_X, batch_y, output)
            
            # Calculate metrics on full dataset
            output_full = self.feedforward(X_tr)
            train_loss = binary_cross_entropy(y_tr, output_full)  # Changed to BCE
            train_acc = np.mean((output_full > 0.5).astype(int) == y_tr) * 100
            
            output_test = self.feedforward(X_te)
            test_loss = binary_cross_entropy(y_te, output_test)  # Changed to BCE
            test_acc = np.mean((output_test > 0.5).astype(int) == y_te) * 100
            
            self.train_loss.append(train_loss)
            self.test_loss.append(test_loss)
            self.train_acc.append(train_acc)
            self.test_acc.append(test_acc)
            
            print(f'Epoch {epoch + 1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}%')
            
            if test_loss < best_loss:
                best_loss = test_loss
                patience_count = 0
            else:
                patience_count += 1
            
            if patience_count >= patience:
                print(f'[Training Stopped] Patience {patience} reached')
                break
            
            if train_loss <= max_error:
                print(f'[Training Stopped] Max error {max_error} reached')
                break
            
        return self.history

    def predict(self, X):
        X_vals = X.values if isinstance(X, pd.DataFrame) else X
        #forward pass
        h_input = np.dot(X_vals, self.weight1) + self.bias1
        h_output = relu(h_input)
        o_input = np.dot(h_output, self.weight2) + self.bias2
        probs = sigmoid(o_input)
        # Return 0 or 1
        return (probs > 0.5).astype(int).flatten()
    

5.2 Initialize, Train and Predict

* Input Layer : 12 nodes
* Hidden Layer : 8 nodes
* Learning Rate : 0.01
* Epochs : 500
* Batch Size : 32

In [ ]:

nn = NeuralNetwork(input_dimension= input_dim, hidden_nodes=8, alpha=0.01)
print('Model is trained')
history = nn.train(X_train_final, y_train_final, X_test_scaled, y_test)    
y_hat = nn.predict(X_test_scaled)
input('Model Trained. Enter to continue to Visualization')


Model is trained
Epoch 1/500 | Train Loss: 0.6661 | Train Acc: 55.56% | Test Loss: 0.5651 | Test Acc: 84.71%
Epoch 2/500 | Train Loss: 0.6508 | Train Acc: 55.59% | Test Loss: 0.5433 | Test Acc: 84.89%
Epoch 3/500 | Train Loss: 0.6357 | Train Acc: 57.35% | Test Loss: 0.5277 | Test Acc: 85.24%
Epoch 4/500 | Train Loss: 0.6190 | Train Acc: 61.80% | Test Loss: 0.5121 | Test Acc: 84.53%
Epoch 5/500 | Train Loss: 0.6004 | Train Acc: 62.92% | Test Loss: 0.4957 | Test Acc: 85.06%
Epoch 6/500 | Train Loss: 0.5812 | Train Acc: 67.31% | Test Loss: 0.4832 | Test Acc: 84.53%
Epoch 7/500 | Train Loss: 0.5615 | Train Acc: 71.90% | Test Loss: 0.4622 | Test Acc: 84.89%
Epoch 8/500 | Train Loss: 0.5432 | Train Acc: 74.27% | Test Loss: 0.4495 | Test Acc: 84.89%
Epoch 9/500 | Train Loss: 0.5264 | Train Acc: 75.74% | Test Loss: 0.4293 | Test Acc: 85.24%
Epoch 10/500 | Train Loss: 0.5112 | Train Acc: 76.47% | Test Loss: 0.4167 | Test Acc: 84.53%
Epoch 11/500 | Train Loss: 0.4971 | Train Acc: 77.04% | Test L

In [ ]:
def evaluate_model_performance(history, y_true, y_pred, title):
    # Create a figure with 3 subplots in 1 row
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle(f'Model Evaluation: {title}', fontsize=16, fontweight='bold')

    # --- 1. Loss Curve Plot ---
    train_loss = history['train_loss']
    val_loss = history['test_loss']
    epochs = range(1, len(train_loss) + 1)

    axes[0].plot(epochs, train_loss, label='Training Loss', color='blue', lw=1.5, marker='x', ms=4)
    axes[0].plot(epochs, val_loss, label='Validation Loss', color='orange', lw=1.5, marker='x', ms=4)
    axes[0].set_title('Loss Curve (Learning Curve)')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # --- 2. Accuracy Plot ---
    train_acc = history['train_acc']
    val_acc = history['test_acc']

    axes[1].plot(epochs, train_acc, label='Training Accuracy', color='blue', lw=1.5, marker='x', ms=4)
    axes[1].plot(epochs, val_acc, label='Validation Accuracy', color='orange', lw=1.5, marker='x', ms=4)
    axes[1].set_title('Accuracy Over Epochs')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    # --- 3. Confusion Matrix ---
    actual = np.array(y_true).flatten()
    predicted = np.array(y_pred).flatten()
    
    tp = np.sum((actual == 1) & (predicted == 1))
    tn = np.sum((actual == 0) & (predicted == 0))
    fp = np.sum((actual == 0) & (predicted == 1))
    fn = np.sum((actual == 1) & (predicted == 0))
    
    cm = np.array([[tn, fp], [fn, tp]])
    cm_df = pd.DataFrame(cm, index=['Actual Stay', 'Actual Churn'], columns=['Predicted Stay', 'Predicted Churn'])
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Stay', 'Churn'], 
            yticklabels=['Stay', 'Churn'])
    axes[2].set_title('Confusion Matrix: Churn Prediction')
    axes[2].set_ylabel('Actual')
    axes[2].set_xlabel('Predicted')
    plt.show()
    # Adjust layout to prevent overlap
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()